In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
from torchmetrics.classification import MulticlassAccuracy

In [3]:
train_data = datasets.MNIST(root=r"C:\Users\Admin\Desktop\Personal\VS Code\Python\PyTorch\Datasets\MNIST",download=True,train=True,transform=ToTensor())
test_data = datasets.MNIST(root=r"C:\Users\Admin\Desktop\Personal\VS Code\Python\PyTorch\Datasets\MNIST",download=True,train=False,transform=ToTensor())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_SIZE = 32

train_dataloader = DataLoader(train_data,shuffle=True,batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data,drop_last=True,batch_size=BATCH_SIZE)

INPUT_FEATURES = 1
HIDDEN_FEATURES = 32
OUTPUT_FEATURES = len(train_data.classes)

class MNISTCNN(nn.Module):
    def __init__(self,in_features, hidden_features, out_features):
        super().__init__() #input shape = [32,1,28,28]
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_features,hidden_features,kernel_size=3,padding=1,stride=1), #[32,32,28,28]
            nn.LeakyReLU(),
            nn.Conv2d(hidden_features,hidden_features,kernel_size=3,padding=1,stride=1), #[32,32,28,28]
            nn.LeakyReLU(),
            nn.MaxPool2d(2,2), #[32,32,14,14]
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(hidden_features,hidden_features,kernel_size=3,padding=1,stride=1),
            nn.LeakyReLU(),
            nn.Conv2d(hidden_features,hidden_features,kernel_size=3,padding=1,stride=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2,2), #[32,32,7,7]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(), # [32,]
            nn.Linear(in_features=hidden_features*7*7,out_features=out_features), #[32,10]
        )

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.classifier(x)
        return x
    

model = MNISTCNN(INPUT_FEATURES,HIDDEN_FEATURES,OUTPUT_FEATURES)

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=1e-1)
accuracy = MulticlassAccuracy(num_classes=len(train_data.classes))

epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = 0,0
    model.train()
    for idx, (x_train, y_train) in enumerate(train_dataloader):
        x_train, y_train = x_train.to(device), y_train.to(device)
        train_logits = model(x_train)
        train_preds = train_logits.argmax(dim=1).squeeze()
        loss = loss_fn(train_logits, y_train)
        train_loss += loss
        acc = accuracy.update(train_preds,y_train)
        train_acc += acc

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    print(f"Epoch : {epoch} | Train loss : {train_loss:.5f} | Train acc : {train_acc:.2f}%" )

MulticlassAccuracy()